<a href="https://colab.research.google.com/github/kio2317/-Kotlin-grammar/blob/main/%EB%91%90%EA%B7%BC%EB%91%90%EA%B7%BC%ED%83%80%EC%9A%B4_%EC%9E%91%EB%AC%BC_%ED%83%80%EC%9D%B4%EB%A8%B8v2_0%EB%B3%B5%EC%82%AC%EB%B3%B8_%EC%A0%9C%EC%9E%91%EC%9E%90_%EC%9C%A0%EB%A6%AC%EC%9C%A0%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#작성자: 유리유리 uid ggbdcgf
이 파일은 복사하셔서 사용하세요~
## 복사방법
### 우측 상단 파일 누른후  Drive 사본 저장 해서 쓰시면 되요.
사용법 가이드 공유 https://docs.google.com/presentation/d/1ixgvkp_VBKhJL1jRww4wS_wljpaW_daQ1YYnuBERcJk/edit?usp=sharing
---
    DISCORD_WEBHOOK_URL = ""
    DISCORD_USER_ID = ""
---
이 줄만 채우면 프로그램 돌릴 수 있어요

# 필수 1회 설치

In [ ]:
!pip -q install ipywidgets

# 프로그램 실행

In [ ]:
# @title {"display-mode":"both"}
import time
import threading
import queue
import requests
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import ipywidgets as widgets
from IPython.display import display, clear_output

# =========================
# 🔧 설정 (여기만 채우면 됨)
# =========================
DISCORD_WEBHOOK_URL = ""
DISCORD_USER_ID = ""

KST = ZoneInfo("Asia/Seoul")

"""
(작물명, 초, 표시용 시간문구)
15초 (15)
15분 (15 * 60)
1시간 (1 * 60 * 60,)
"""


CROPS = {
    "test": ("🧪 테스트 씨앗", 15, "15초"),
    "tomato": ("🍅 토마토 씨앗", 15 * 60, "15분"),
    "potato": ("🥔 감자 씨앗", 1 * 60 * 60, "60분"),
    "wheat": ("🌾 밀 씨앗", 4 * 60 * 60, "4시간"),
    "lettuce": ("🥬 상추 씨앗", 8 * 60 * 60, "8시간"),
    "pineapple": ("🍍 파인애플 씨앗", 30 * 60, "30분"),
    "carrot": ("🥕 당근 씨앗", 2 * 60 * 60, "2시간"),
    # 작물 추가하고 싶은거 추가하시면 됩니다.
}

# @@@@아래는 건드리지 마세요.@@@@
# 개인이 직접 직접 다룰 수 있으면 건드려도 됩니다.

def format_korean_ampm(dt: datetime) -> str:
    hour = dt.hour
    minute = dt.minute
    ampm = "오전" if hour < 12 else "오후"
    h12 = hour % 12
    if h12 == 0:
        h12 = 12
    return f"{ampm} {h12}:{minute:02d}"

# =========================
# ✅ 디스코드 전송
# =========================
send_queue = queue.Queue()

def discord_sender():
    while True:
        msg = send_queue.get()
        try:
            while True:
                r = requests.post(DISCORD_WEBHOOK_URL, json={"content": msg}, timeout=20)
                if r.status_code in (200, 204):
                    break
                if r.status_code == 429:
                    retry = r.json().get("retry_after", 1)
                    time.sleep(retry)
                    continue
                raise RuntimeError(f"Discord webhook failed: {r.status_code} {r.text}")
            # 한 번 보낸 후 살짝 쉬기 (연속 방지)
            time.sleep(0.6)
        finally:
            send_queue.task_done()

def send_discord(msg: str):
    send_queue.put(msg)

# 워커 시작(백그라운드)
threading.Thread(target=discord_sender, daemon=True).start()

# =========================
# ✅ 작물 타이머(멀티)
# =========================
def run_crop(crop_name: str, seconds: int, label: str):
    now = datetime.now(KST)
    end_time = now + timedelta(seconds=seconds)
    end_str = format_korean_ampm(end_time)

    pretty_name = f"{crop_name}({label})"

    # 심기 완료
    send_discord(f"✅ {pretty_name} 심기 완료!\n⏳ 예상 수확: {end_str}")

    time.sleep(seconds)

    # 수확(원하면 여기 2줄 포맷으로 바꿔도 됨)
    send_discord(f"✅ 수확 시간! {pretty_name} 지금 수확하러 가자 🌱 <@{DISCORD_USER_ID}>")

# =========================
# 🌱 UI (예쁜 입력)
# =========================
title = widgets.HTML(
    value="<h3 style='margin:6px 0'>🌱 두근두근타운 밭 알림</h3>"
)
subtitle = widgets.HTML(
    value="<div style='color:#666; margin-bottom:6px'>심을 작물을 선택하고 <b>🌱 심기</b> 버튼을 눌러줘</div>"
)

options = []
for key, (name, _, label) in CROPS.items():
    options.append((f"{name}({label})", key))

select = widgets.SelectMultiple(
    options=options,
    value=(),
    rows=6,
    description="작물",
    layout=widgets.Layout(width="420px", height="180px"),
)

start_btn = widgets.Button(
    description="🌱 심기",
    button_style="success",
    layout=widgets.Layout(width="120px", height="40px"),
)
clear_btn = widgets.Button(
    description="🧹 출력 지우기",
    button_style="",
    layout=widgets.Layout(width="120px", height="40px"),
)

status = widgets.HTML(value="")
out = widgets.Output(layout=widgets.Layout(border="1px solid #eee", padding="10px", width="560px"))

def on_clear(_):
   with out:
    clear_output()  # ✅ 심기 누를 때마다 출력창 자동 초기화
    status.value = ""

clear_btn.on_click(on_clear)

def on_start(_):
    picked = list(select.value)
    if not picked:
        status.value = "<b style='color:#d00'>작물을 하나 이상 선택해줘!</b>"
        return

    now = datetime.now(KST)
    now_str = format_korean_ampm(now)
    status.value = f"<b>현재시각:</b> {now_str} · <b>선택:</b> {len(picked)}개"

    # ✅ 여기서 출력창을 먼저 비워서 "현재 실행"만 보이게
    with out:
        clear_output()

        print("두근두근타운 작물 타이머 🌱")
        print(f"현재시각 {now_str}\n")
        print("🌱 심은 작물:")

        # 코랩 목록에 (예상 수확) 표시
        for key in picked:
            name, sec, label = CROPS[key]
            end_str = format_korean_ampm(now + timedelta(seconds=sec))
            print(f"- {name}({label})")
            print(f"  (예상 수확 {end_str})")

        print("\n✅ 타이머 시작! 디스코드 알림이 순차적으로 전송돼요.")

    # 타이머 스레드 시작
    threads = []
    for key in picked:
        name, sec, label = CROPS[key]
        t = threading.Thread(target=run_crop, args=(name, sec, label), daemon=True)
        t.start()
        threads.append(t)

    # 완료 메시지는 코랩에서만
    def wait_and_finish():
        for t in threads:
            t.join()
        send_queue.join()
        with out:
            print("\n✅ 모든 작물 수확 완료!")

    threading.Thread(target=wait_and_finish, daemon=True).start()


start_btn.on_click(on_start)

controls = widgets.HBox([start_btn, clear_btn], layout=widgets.Layout(gap="10px"))
ui = widgets.VBox([title, subtitle, select, controls, status, out])
display(ui)
